In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

Checking that I can get example problem from cvxpy documentation (https://www.cvxpy.org/examples/basic/mixed_integer_quadratic_program.html#basic-examples) to run:

In [9]:
# Generate a random problem
np.random.seed(0)
m, n= 40, 25

A = np.random.rand(m, n)
b = np.random.randn(m)

# Construct a CVXPY problem
x = cp.Variable(n, integer=True)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
prob = cp.Problem(objective)
prob.solve()

13.66000325196205

In [10]:
print("Status: ", prob.status)
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)

Status:  optimal
The optimal value is 13.66000325196205
A solution x is
[-1.  1.  1. -1.  0.  0. -1. -2.  0.  0.  0.  1.  1.  0.  1.  0. -1. -1.
 -1.  0.  2. -1.  2.  0. -1.]


Now I'll adapt the example to match my problem:

In [30]:
#User Inputs:
#Dimensions of BOM items:
a = np.array([5,5,10,9,10]) #longest dimension
b = np.array([4,5,1,5,5]) #mid dimension
c = np.array([0.5,0.5,0.5,0.5,0.5]) #smallest dimension

#Dimensions of UNIQUE stock items available for purchase:
l = np.array([15,15]) #longest dimension
w = np.array([10,15]) #mid dimension
h = np.array([0.5,0.5]) #smallest dimension

#Prices of UNIQUE stock items available for purchase:
prices = [30,40]

#Dimension info:
n = len(a) #Total number of items on BOM
m = n * len(l) #Upper limit of stock items we'll have to buy

#Adjust dimensions of p / replicate each stock board n-1 times:
p = []
for price in prices:
    for i in range(n):
        p.append(price)

p = np.array(p)

In [31]:
#Decision Variables:
u = cp.Variable((n,m),boolean=True)
x = cp.Variable((n,),nonneg=True)
y = cp.Variable((n,),nonneg=True)
r = cp.Variable((n,),boolean=True)

In [32]:
#Intermediate Variables:
q = cp.Variable((m,),boolean=True)
s = cp.Variable((n,n),boolean=True)
t = cp.Variable((n,n),boolean=True)
v = cp.Variable((n,n),boolean=True)
d = cp.Variable((n,n),boolean=True)
f = cp.Variable((n,n),boolean=True)
g = cp.Variable((n,n),boolean=True)

In [33]:
#Objective Function:
objective = cp.Minimize(cp.sum(cp.multiply(p, q)))

In [34]:
#Constraints:
constraints = [
    cp.sum(u, axis=1) == 1, #constr. 1
    cp.sum(u <= c/h) == n*m, #2a
    cp.sum(u <= h/c) == n*m, #2b
    q >= cp.sum(u, axis=0) / n, #3
    v == s + t #4a
]

#https://stackoverflow.com/questions/53839771/how-to-tell-cvxpy-how-to-sum-over-values-of-a-matrix-in-an-objective-function:
for i in range(n):
    for k in range(n):
        if i != k:
            constraints.append(s[i,k] <= (y[k] + 1) / (y[i] + (1-r[i])*a[i] + r[i]*b[i] + 1))

ValueError: operands could not be broadcast together with shapes (5,) (2,) 

In [ ]:
problem = cp.Problem(objective, constraints)
problem.solve()
print("Optimal value: ",problem.value)
print("Optimal u: ",u.value)
print("Optimal x: ",x.value)
print("Optimal y: ",y.value)
print("Optimal r: ",r.value)